In [59]:
# The code was removed by Watson Studio for sharing.

# Coursera IBM Data Science Course 9 Applied Data Science Capstone project - Vancouver

In [114]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Obtained geospatial data from https://www.geonames.org

In [97]:
# access csv file saved in IBM Watson Studio project directory
print('Project Name: {0}'.format(project.get_name()))
print('Project Assets: {0}'.format(project.get_assets()))
pd.read_csv(project.get_file('CA.csv')).head()

Project Name: 9_Applied Data Science Capstone
Project Assets: [{'name': '9_Applied Data Science Capstone Project Notebook T', 'asset_id': 'f96c884d-968b-4cca-a77d-720b61262b59', 'type': 'notebook'}, {'name': '9_Applied Data Science Capstone Project Notebook V', 'asset_id': '28b23357-f7bc-4dfa-8108-ca91633c2c4f', 'type': 'notebook'}, {'name': 'CA.csv', 'asset_id': '653ca424-f846-4ff1-a5d6-12e82f547e2c', 'type': 'data_asset'}]


,country code,postal code,place name,admin name1,admin code1,admin name2,admin code2,admin name3,admin code3,latitude,longitude,accuracy
0,CA,T0A,Eastern Alberta (St. Paul),Alberta,AB,NaN,NaN,NaN,NaN,54.7660,-111.7174,6.0
1,CA,T0B,Wainwright Region (Tofield),Alberta,AB,NaN,NaN,NaN,NaN,53.0727,-111.5816,6.0
2,CA,T0C,Central Alberta (Stettler),Alberta,AB,NaN,NaN,NaN,NaN,52.1431,-111.6941,5.0
3,CA,T0E,Western Alberta (Jasper),Alberta,AB,NaN,NaN,NaN,NaN,53.6758,-115.0948,5.0
4,CA,T0G,North Central Alberta (Slave Lake),Alberta,AB,NaN,NaN,NaN,NaN,55.6993,-114.4529,6.0


In [119]:
df=pd.read_csv(project.get_file('CA.csv'))[['postal code','place name','latitude','longitude']]
df.rename(columns={'postal code':'Postal Code','place name':'Neighborhood','latitude':'Latitude','longitude':'Longitude'}, inplace=True) # rename column
van_data=df[df['Postal Code'].str.contains('V3|V4|V5|V6|V7')].reset_index(drop=True) # only keep Great Vancouver data
van_data.head()

,Postal Code,Neighborhood,Latitude,Longitude
0,V5K,Vancouver (North Hastings-Sunrise),49.2807,-123.0397
1,V5L,Vancouver (North Grandview-Woodlands),49.2795,-123.0667
2,V5M,Vancouver (South Hastings-Sunrise / North Renf...,49.2600,-123.0398
3,V5N,Vancouver (South Grandview-Woodlands / NE Kens...,49.2551,-123.0667
4,V5P,Vancouver (SE Kensington / Victoria-Fraserview),49.2220,-123.0683


#### Use geopy library to get the latitude and longitude values of Vancouver.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>tor_explorer</em>, as shown below.

In [120]:
address = 'Vancouver, BC'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


#### Create a map of Vancouver with neighborhoods superimposed on top.

In [122]:
# create map of Vancouver using latitude and longitude values
map_van = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(van_data['Latitude'], van_data['Longitude'], van_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_van)  
    
map_van

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [123]:
CLIENT_ID = 'KAQ4OMOMOYVRGLLB4V0GWGNTXLRJMIGWQXFDKXCA5XLYFKTD' # your Foursquare ID
CLIENT_SECRET = 'HXIMNFZJSSOW5BPCJSBTUNZYUONC0OC0NQD35FXF1KNXFZYS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KAQ4OMOMOYVRGLLB4V0GWGNTXLRJMIGWQXFDKXCA5XLYFKTD
CLIENT_SECRET:HXIMNFZJSSOW5BPCJSBTUNZYUONC0OC0NQD35FXF1KNXFZYS


#### Let's create a function to get top 20 venues within 1000-meter radius of each neighborhood in Vancouver

In [247]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, limit=20):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe

In [248]:
van_venues = getNearbyVenues(names=van_data['Neighborhood'],
                            latitudes=van_data['Latitude'],
                            longitudes=van_data['Longitude']
                            )

Vancouver (North Hastings-Sunrise)
Vancouver (North Grandview-Woodlands)
Vancouver (South Hastings-Sunrise / North Renfrew-Collingwood)
Vancouver (South Grandview-Woodlands / NE Kensington)
Vancouver (SE Kensington / Victoria-Fraserview)
Vancouver (South Renfrew-Collingwood)
Vancouver (Killarney)
Vancouver (East Mount Pleasant)
Vancouver (West Kensington / NE Riley Park-Little Mountain)
Vancouver (SE Riley Park-Little Mountain / SW Kensington / NE Oakridge / North Sunset)
Vancouver (SE Oakridge / East Marpole / South Sunset)
Vancouver (West Mount Pleasant / West Riley Park-Little Mountain)
Vancouver (East Fairview / South Cambie)
Vancouver (Strathcona / Chinatown / Downtown Eastside)
Vancouver (NE Downtown / Harbour Centre / Gastown / Yaletown)
Vancouver (Waterfront / Coal Harbour / Canada Place)
Vancouver (South West End)
Vancouver (North West End / Stanley Park)
Vancouver (West Fairview / Granville Island / NE Shaughnessy)
Vancouver (NW Shaughnessy / East Kitsilano / Quilchena)
Vanco

In [249]:
print(van_venues.shape)
van_venues.head()

(1223, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,The Fair at the PNE,49.282971,-123.042109,Fair
1,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Wooden Roller Coaster,49.281744,-123.035128,Theme Park Ride / Attraction
2,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Livestock Barns,49.284037,-123.039278,Farm
3,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Playland,49.281924,-123.036258,Theme Park
4,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Tamam Fine Palestinian Cuisine,49.281070,-123.051438,Middle Eastern Restaurant


Let's check how many venues were returned for each neighborhood

In [250]:
van_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Burnaby (Burnaby Heights / Willingdon Heights / West Central Valley),20,20,20,20,20,20
Burnaby (Cascade-Schou / Douglas-Gilpin),11,11,11,11,11,11
Burnaby (East Big Bend / Stride Avenue / Edmonds / Cariboo-Armstrong),11,11,11,11,11,11
Burnaby (Government Road / Lake City / SFU / Burnaby Mountain),9,9,9,9,9,9
Burnaby (Lakeview-Mayfield / Richmond Park / Kingsway-Beresford),20,20,20,20,20,20
Burnaby (Maywood / Marlborough / Oakalla / Windsor),20,20,20,20,20,20
Burnaby (Parkcrest-Aubrey / Ardingley-Sprott),20,20,20,20,20,20
Burnaby (Suncrest / Sussex-Nelson / Clinton-Glenwood / West Big Bend),19,19,19,19,19,19
Coquitlam North,6,6,6,6,6,6


In [251]:
print('There are {} uniques categories.'.format(len(van_venues['Venue Category'].unique())))

There are 219 uniques categories.


In [252]:
# one hot encoding
van_onehot = pd.get_dummies(van_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
van_onehot['Neighborhood'] = van_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [van_onehot.columns[-1]] + list(van_onehot.columns[:-1])
van_onehot = van_onehot[fixed_columns]

van_onehot.head()

,Neighborhood,ATM,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Aquarium,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Garden,Belgian Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drive-in Theater,Electronics Store,Elementary School,Event Space,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Noodle House,Office,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chairlift,Ski Chalet,Ski Lodge,Ski Trail,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Spa,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trade School,Trail,Train Station,Travel Lounge,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,

In [253]:
# examine dataframe size
van_onehot.shape

(1223, 220)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [254]:
van_grouped = van_onehot.groupby('Neighborhood').mean().reset_index()
van_grouped

,Neighborhood,ATM,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Aquarium,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Garden,Belgian Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drive-in Theater,Electronics Store,Elementary School,Event Space,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Noodle House,Office,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chairlift,Ski Chalet,Ski Lodge,Ski Trail,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Spa,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trade School,Trail,Train Station,Travel Lounge,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Burnaby (Burnaby Heights / Willingdon Heights ...,0.00,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.050000,0.00,0.050000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.05,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.05,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.100000,0.00,0.00,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.05,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.050000,0.050000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.050000,0.000000,0.00,0.00,0.000000,0.0,0.00,0.000000,0.00,0.000,0.00,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.

In [255]:
# confirm new dataframe size
van_grouped.shape

(88, 220)

Function to sort each neighborhood's venues in descending order.

In [256]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [271]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = van_grouped['Neighborhood']

for ind in np.arange(van_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(van_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Burnaby (Burnaby Heights / Willingdon Heights ...,Café,French Restaurant,Sushi Restaurant,Sporting Goods Shop,Liquor Store,Restaurant,Brewery,Greek Restaurant,Tea Room,Thai Restaurant
1,Burnaby (Cascade-Schou / Douglas-Gilpin),Sandwich Place,Insurance Office,Vegetarian / Vegan Restaurant,Park,Auto Garage,Cafeteria,Sushi Restaurant,Snack Place,Chinese Restaurant,Food & Drink Shop
2,Burnaby (East Big Bend / Stride Avenue / Edmon...,Market,Park,Burger Joint,Chinese Restaurant,Motorcycle Shop,Sandwich Place,Convenience Store,Art Gallery,Deli / Bodega,Italian Restaurant
3,Burnaby (Government Road / Lake City / SFU / B...,Burger Joint,Café,Golf Course,Golf Driving Range,Pizza Place,Baseball Field,Grocery Store,Sporting Goods Shop,Yoga Studio,Filipino Restaurant
4,Burnaby (Lakeview-Mayfield / Richmond Park / K...,Sushi Restaurant,Yoga Studio,Liquor Store,Bus Stop,Frozen Yogurt Shop,Chinese Restaurant,Breakfast Spot,Shopping Mall,Middle Eastern Restaurant,Fast Food Restaurant
5,Burnaby (Maywood / Marlborough / Oakalla / Win...,Bookstore,Portuguese Restaurant,Fast Food Restaurant,Shanghai Restaurant,Shopping Mall,Grocery Store,Clothing Store,Bakery,Toy / Game Store,Electronics Store
6,Burnaby (Parkcrest-Aubrey / Ardingley-Sprott),Italian Restaurant,Bank,Coffee Shop,Pizza Place,Pharmacy,Shopping Mall,Fish & Chips Shop,Café,Burger Joint,Breakfast Spot
7,Burnaby (Suncrest / Sussex-Nelson / Clinton-Gl...,Garden Center,Golf Course,Restaurant,Coffee Shop,Pet Store,Men's Store,Women's Store,Baseball Field,Shopping Mall,American Restaurant
8,Coquitlam North,Bus Stop,Park,Business Service,Convenience Store,Falafel Restaurant,Yoga Studio,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
9,Coquitlam South,Furniture / Home Store,Comfort Food Restaurant,Discount Store,Sushi Restaurant,Greek Restaurant,Baseball Field,Grocery Store,Food Truck,Sandwich Place,Lounge


Run *k*-means to cluster the neighborhoods into 5 clusters.

In [272]:
# set number of clusters
kclusters = 6

van_grouped_clustering = van_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(van_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 1, 3, 3, 3, 1, 3, 3, 3, 0, 3],
      dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [273]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

van_merged = van_data

# merge grouped with data to add latitude/longitude for each neighborhood
van_merged = van_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

van_merged.head() # check the last columns!

,Postal Code,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V5K,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,3.0,Vietnamese Restaurant,Theme Park Ride / Attraction,Park,Coffee Shop,Sandwich Place,Farm,Theme Park,Food Truck,Bakery,Middle Eastern Restaurant
1,V5L,Vancouver (North Grandview-Woodlands),49.2795,-123.0667,3.0,Brewery,Café,Theater,Coffee Shop,Breakfast Spot,Fried Chicken Joint,French Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant
2,V5M,Vancouver (South Hastings-Sunrise / North Renf...,49.2600,-123.0398,3.0,Coffee Shop,Hardware Store,Grocery Store,Pizza Place,Pet Store,Burger Joint,Paper / Office Supplies Store,Office,Liquor Store,Sandwich Place
3,V5N,Vancouver (South Grandview-Woodlands / NE Kens...,49.2551,-123.0667,3.0,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Café,Vietnamese Restaurant,Park,Music Venue,Farmers Market,Grocery Store,Bakery,Gym
4,V5P,Vancouver (SE Kensington / Victoria-Fraserview),49.2220,-123.0683,3.0,Pizza Place,Convenience Store,Gas Station,Sandwich Place,Thrift / Vintage Store,Bank,Motorcycle Shop,Flower Shop,Noodle House,Vietnamese Restaurant


Finally, let's visualize the resulting clusters

In [274]:
van_merged_dropna=van_merged.dropna()
van_merged_dropna.groupby('Cluster Labels').count()

,Postal Code,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,
0.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1.0,10,10,10,10,10,10,10,10,10,10,10,10,10,10
2.0,5,5,5,5,5,5,5,5,5,5,5,5,5,5
3.0,63,63,63,63,63,63,63,63,63,63,63,63,63,63
4.0,6,6,6,6,6,6,6,6,6,6,6,6,6,6
5.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [275]:
van_merged_dropna['Cluster Labels']=van_merged_dropna['Cluster Labels'].astype(int)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [276]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(van_merged_dropna['Latitude'], van_merged_dropna['Longitude'], van_merged_dropna['Neighborhood'], van_merged_dropna['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters